# Crashes to JSON

Use `Data Query Tool` to create a new export from UCONN Crashes (Town=Hartford, last five years): https://www.ctcrash.uconn.edu/QueryTool2.action

* Since the tool does not let you export large datasets (over 90,000 persons), export multiple times for different time periods and specify all export IDs in the `exports` list below.

This notebook combines data from two tables (0 and 2) into a small JSON file and a CSV file (same content, different formats), which is an array of objects that look like this:
```js
{
    "id": 770826,    // crash id (used to pull the diagram)
    "x": 41.73936,   // latitude
    "y": -72.66404,  // longitude
    "d": 1593820800, // timestamp
    "p": 0,          // involves a pedestrian? (1=true, 0=false)
    "c": 0,          // involves a cyclist? (1=true, 0=false)
    "r": 1,          // route class (1=Interstate, 2=USRoute, 3=State, 4=Local)
    "s": 0,          // crash severity (O=no injuries, A=any injury, K=fatal)
}
```

Note: the timestamp `d` needs to be multiplied by `100,000` for a proper JS timestamp.

In [14]:
import pandas as pd

In [15]:
exports = ['165957']

### Import roles from table 2

In [16]:
roles = []

for export in exports:
    df = pd.read_csv(
        f'export_{export}/export_{export}_2.csv',
        skiprows=1,
        usecols=['CrashId', 'Person Type Text Format']
    )
    roles.append( df )

roles_df = pd.concat(roles).groupby(['CrashId'])['Person Type Text Format'].unique() # .groupby(['CrashId'])['Person Type Text Format'].unique()


### Import crashes from table 0

In [17]:
localData = []

for export in exports:
    df = pd.read_csv(
        f'export_{export}/export_{export}_0.csv',
        skiprows=1,
        usecols=['CrashId', 'Latitude', 'Longitude', 'Date Of Crash',
                 'Time of Crash', 'Crash Severity', 'Route Class',
                 'Roadway Name', 'Intersecting Roadway Name']
    )
    localData.append( df )

localData_df = pd.concat(localData)

crashesTable0_df = localData_df.merge(roles_df, on='CrashId', how='inner')

crashesTable0_df.head()

,CrashId,Latitude,Longitude,Date Of Crash,Time of Crash,Crash Severity,Route Class,Roadway Name,Intersecting Roadway Name,Person Type Text Format
0,4124,41.334265,-72.900609,2015-02-06,12:40:00,A,2.0,STATE STREET,NaN,[Pedestrian]
1,8555,41.347714,-72.934389,2015-03-27,19:59:00,A,4.0,CIRCULAR AVE,GORHAM AV,[Pedestrian]
2,29371,41.314237,-72.943333,2015-01-14,08:53:00,A,4.0,SHERMAN AVE,MAPLE ST,[Driver]
3,30371,41.304200,-72.930900,2015-02-17,17:01:00,A,4.0,COLLEGE ST,NORTH FRONTAGE RD,[Pedestrian]
4,31050,41.313823,-72.925020,2015-03-07,16:30:00,A,4.0,TRUMBULL ST,PROSPECT ST,[Bicyclist]


Import crashes from table 1


In [18]:
vehicleData = []

for export in exports:
    df = pd.read_csv(
        f'export_{export}/export_{export}_1.csv',
        skiprows = 1,
        usecols=['CrashId', 'Direction of Travel Before Crash', 'Contributing Circumstances, Motor Vehicle', 
                 'Posted/Statutory Speed Limit', 'Bike Lanes/Sharrows Present', 'Hit And Run Status']
    )
    vehicleData.append ( df )

crashesTable1_df = pd.concat(vehicleData)

crashesTableFinal_df = crashesTable0_df.merge(crashesTable1_df, on='CrashId', how='inner')

### Merge data and save to .json and .csv

In [19]:
crashesTableFinal_df.to_csv('crashes.csv', index=False)

crashesTableFinal_df.head()


,CrashId,Latitude,Longitude,Date Of Crash,Time of Crash,Crash Severity,Route Class,Roadway Name,Intersecting Roadway Name,Person Type Text Format,Direction of Travel Before Crash,"Contributing Circumstances, Motor Vehicle",Posted/Statutory Speed Limit,Hit And Run Status,Bike Lanes/Sharrows Present
0,4124,41.334265,-72.900609,2015-02-06,12:40:00,A,2.0,STATE STREET,NaN,[Pedestrian],S,0,35.0,False,False
1,8555,41.347714,-72.934389,2015-03-27,19:59:00,A,4.0,CIRCULAR AVE,GORHAM AV,[Pedestrian],E,99,25.0,True,NaN
2,29371,41.314237,-72.943333,2015-01-14,08:53:00,A,4.0,SHERMAN AVE,MAPLE ST,[Driver],N,0,NaN,True,True
3,30371,41.304200,-72.930900,2015-02-17,17:01:00,A,4.0,COLLEGE ST,NORTH FRONTAGE RD,[Pedestrian],W,0,25.0,False,False
4,31050,41.313823,-72.925020,2015-03-07,16:30:00,A,4.0,TRUMBULL ST,PROSPECT ST,[Bicyclist],N,0,25.0,False,False


In [20]:
# Rename columns to make JSON output smaller
crashesTableFinal_df.columns = ['id', 'x', 'y', 'd', 't', 's', 'r', 'o', 'h',\
    'k', 'j', 'm', 'b', 'v', 'f']

crashesTableFinal_df.head()

,id,x,y,d,t,s,r,o,h,k,j,m,b,v,f
0,4124,41.334265,-72.900609,2015-02-06,12:40:00,A,2.0,STATE STREET,NaN,[Pedestrian],S,0,35.0,False,False
1,8555,41.347714,-72.934389,2015-03-27,19:59:00,A,4.0,CIRCULAR AVE,GORHAM AV,[Pedestrian],E,99,25.0,True,NaN
2,29371,41.314237,-72.943333,2015-01-14,08:53:00,A,4.0,SHERMAN AVE,MAPLE ST,[Driver],N,0,NaN,True,True
3,30371,41.304200,-72.930900,2015-02-17,17:01:00,A,4.0,COLLEGE ST,NORTH FRONTAGE RD,[Pedestrian],W,0,25.0,False,False
4,31050,41.313823,-72.925020,2015-03-07,16:30:00,A,4.0,TRUMBULL ST,PROSPECT ST,[Bicyclist],N,0,25.0,False,False


In [21]:
# Transform timestamps
crashesTableFinal_df.d = crashesTableFinal_df.d.apply(lambda x: int(pd.to_datetime(x).timestamp() / 100))
crashesTableFinal_df.t = crashesTableFinal_df.t.apply(lambda x: x[:-3])

# Set up pedestrian and cyclist flags
crashesTableFinal_df['p'] = crashesTableFinal_df.id.apply(lambda x: 1 if x in roles_df and 'Pedestrian' in roles_df[x] else 0)
crashesTableFinal_df['c'] = crashesTableFinal_df.id.apply(lambda x: 1 if x in roles_df and 'Bicyclist' in roles_df[x] else 0)

crashesTableFinal_df.head()

,id,x,y,d,t,s,r,o,h,k,j,m,b,v,f,p,c
0,4124,41.334265,-72.900609,14231808,12:40,A,2.0,STATE STREET,NaN,[Pedestrian],S,0,35.0,False,False,1,0
1,8555,41.347714,-72.934389,14274144,19:59,A,4.0,CIRCULAR AVE,GORHAM AV,[Pedestrian],E,99,25.0,True,NaN,1,0
2,29371,41.314237,-72.943333,14211936,08:53,A,4.0,SHERMAN AVE,MAPLE ST,[Driver],N,0,NaN,True,True,0,0
3,30371,41.304200,-72.930900,14241312,17:01,A,4.0,COLLEGE ST,NORTH FRONTAGE RD,[Pedestrian],W,0,25.0,False,False,1,0
4,31050,41.313823,-72.925020,14256864,16:30,A,4.0,TRUMBULL ST,PROSPECT ST,[Bicyclist],N,0,25.0,False,False,0,1


In [22]:
# Save as JSON
crashesTableFinal_df.to_json('./crashes.json', orient='records', double_precision=5)

# Save as CSV
crashesTableFinal_df.to_csv('crashes.csv', index=False)